# Модель линейной регрессии
Данные <br>
http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz <br>
http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz <br>  
http://video.ittensive.com/machine-learning/ashrae/train.0.0.csv.gz    

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn import linear_model 
import matplotlib.pyplot as plt
from matplotlib.pyplot import rcParams
rcParams['figure.figsize'] = 16, 8

In [2]:
# загрузка с локаьного компьютера
full_path = "/home/al/PycharmProjects/examples_problem_solving_kaggl/mashin_learning_and_kaggle/jupiter_bilder/"
buildings = pd.read_csv(full_path + "building_metadata.csv")
print (buildings.head())

   site_id  building_id primary_use  square_feet  year_built  floor_count
0        0            0   Education         7432      2008.0          NaN
1        0            1   Education         2720      2004.0          NaN
2        0            2   Education         5376      1991.0          NaN
3        0            3   Education        23685      2002.0          NaN
4        0            4   Education       116607      1975.0          NaN


In [3]:
weather = pd.read_csv(full_path + "weather_train.csv")
print (weather.head())

   site_id            timestamp  air_temperature  cloud_coverage  \
0        0  2016-01-01 00:00:00             25.0             6.0   
1        0  2016-01-01 01:00:00             24.4             NaN   
2        0  2016-01-01 02:00:00             22.8             2.0   
3        0  2016-01-01 03:00:00             21.1             2.0   
4        0  2016-01-01 04:00:00             20.0             2.0   

   dew_temperature  precip_depth_1_hr  sea_level_pressure  wind_direction  \
0             20.0                NaN              1019.7             0.0   
1             21.1               -1.0              1020.2            70.0   
2             21.1                0.0              1020.2             0.0   
3             20.6                0.0              1020.1             0.0   
4             20.0               -1.0              1020.0           250.0   

   wind_speed  
0         0.0  
1         1.5  
2         0.0  
3         0.0  
4         2.6  


In [4]:
energy_0 = pd.read_csv(full_path + "train.csv")
print (energy_0.head())

   building_id  meter            timestamp  meter_reading
0            0      0  2016-01-01 00:00:00            0.0
1            1      0  2016-01-01 00:00:00            0.0
2            2      0  2016-01-01 00:00:00            0.0
3            3      0  2016-01-01 00:00:00            0.0
4            4      0  2016-01-01 00:00:00            0.0


In [5]:
energy_0 = pd.merge(left=energy_0, right=buildings, how="left",
                   left_on="building_id", right_on="building_id")
print (energy_0.head())

   building_id  meter            timestamp  meter_reading  site_id  \
0            0      0  2016-01-01 00:00:00            0.0        0   
1            1      0  2016-01-01 00:00:00            0.0        0   
2            2      0  2016-01-01 00:00:00            0.0        0   
3            3      0  2016-01-01 00:00:00            0.0        0   
4            4      0  2016-01-01 00:00:00            0.0        0   

  primary_use  square_feet  year_built  floor_count  
0   Education         7432      2008.0          NaN  
1   Education         2720      2004.0          NaN  
2   Education         5376      1991.0          NaN  
3   Education        23685      2002.0          NaN  
4   Education       116607      1975.0          NaN  


In [6]:
energy_0.set_index(["timestamp", "site_id"], inplace=True)
weather.set_index(["timestamp", "site_id"], inplace=True)
energy_0 = pd.merge(left=energy_0, right=weather, how="left", left_index=True, right_index=True)
energy_0.reset_index(inplace=True)
print (energy_0.head())

             timestamp  site_id  building_id  meter  meter_reading  \
0  2016-01-01 00:00:00        0            0      0            0.0   
1  2016-01-01 00:00:00        0            1      0            0.0   
2  2016-01-01 00:00:00        0            2      0            0.0   
3  2016-01-01 00:00:00        0            3      0            0.0   
4  2016-01-01 00:00:00        0            4      0            0.0   

  primary_use  square_feet  year_built  floor_count  air_temperature  \
0   Education         7432      2008.0          NaN             25.0   
1   Education         2720      2004.0          NaN             25.0   
2   Education         5376      1991.0          NaN             25.0   
3   Education        23685      2002.0          NaN             25.0   
4   Education       116607      1975.0          NaN             25.0   

   cloud_coverage  dew_temperature  precip_depth_1_hr  sea_level_pressure  \
0             6.0             20.0                NaN              10

In [7]:
energy_0["air_temperature"].fillna(0, inplace=True)
energy_0["cloud_coverage"].fillna(0, inplace=True)
energy_0["dew_temperature"].fillna(0, inplace=True)
energy_0["precip_depth_1_hr"] = energy_0["precip_depth_1_hr"].apply(lambda x:x if x>0 else 0)
energy_0_sea_level_pressure_mean = energy_0["sea_level_pressure"].mean()
energy_0["sea_level_pressure"] = energy_0["sea_level_pressure"].apply(lambda x:energy_0_sea_level_pressure_mean if x!=x else x)
energy_0_wind_direction_mean = energy_0["wind_direction"].mean()
energy_0["wind_direction"] = energy_0["wind_direction"].apply(lambda x:energy_0_wind_direction_mean if x!=x else x)
energy_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20216100 entries, 0 to 20216099
Data columns (total 16 columns):
 #   Column              Dtype  
---  ------              -----  
 0   timestamp           object 
 1   site_id             int64  
 2   building_id         int64  
 3   meter               int64  
 4   meter_reading       float64
 5   primary_use         object 
 6   square_feet         int64  
 7   year_built          float64
 8   floor_count         float64
 9   air_temperature     float64
 10  cloud_coverage      float64
 11  dew_temperature     float64
 12  precip_depth_1_hr   float64
 13  sea_level_pressure  float64
 14  wind_direction      float64
 15  wind_speed          float64
dtypes: float64(10), int64(4), object(2)
memory usage: 2.4+ GB


In [8]:
# создаем массив только для здания с id = 0
energy_build_0 = energy_0.loc[energy_0['building_id'] == 0]
print (energy_build_0.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8784 entries, 0 to 20213734
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   timestamp           8784 non-null   object 
 1   site_id             8784 non-null   int64  
 2   building_id         8784 non-null   int64  
 3   meter               8784 non-null   int64  
 4   meter_reading       8784 non-null   float64
 5   primary_use         8784 non-null   object 
 6   square_feet         8784 non-null   int64  
 7   year_built          8784 non-null   float64
 8   floor_count         0 non-null      float64
 9   air_temperature     8784 non-null   float64
 10  cloud_coverage      8784 non-null   float64
 11  dew_temperature     8784 non-null   float64
 12  precip_depth_1_hr   8784 non-null   float64
 13  sea_level_pressure  8784 non-null   float64
 14  wind_direction      8784 non-null   float64
 15  wind_speed          8784 non-null   float64
dtypes:

In [9]:
energy_build_0["timestamp"] = pd.to_datetime(energy_build_0["timestamp"])
energy_build_0["hour"] = energy_build_0["timestamp"].dt.hour
print (energy_build_0.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8784 entries, 0 to 20213734
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   timestamp           8784 non-null   datetime64[ns]
 1   site_id             8784 non-null   int64         
 2   building_id         8784 non-null   int64         
 3   meter               8784 non-null   int64         
 4   meter_reading       8784 non-null   float64       
 5   primary_use         8784 non-null   object        
 6   square_feet         8784 non-null   int64         
 7   year_built          8784 non-null   float64       
 8   floor_count         0 non-null      float64       
 9   air_temperature     8784 non-null   float64       
 10  cloud_coverage      8784 non-null   float64       
 11  dew_temperature     8784 non-null   float64       
 12  precip_depth_1_hr   8784 non-null   float64       
 13  sea_level_pressure  8784 non-null   float64 

<ipython-input-9-ef7af3ef2549>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  energy_build_0["timestamp"] = pd.to_datetime(energy_build_0["timestamp"])
<ipython-input-9-ef7af3ef2549>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  energy_build_0["hour"] = energy_build_0["timestamp"].dt.hour


In [10]:
from sklearn.model_selection import train_test_split

In [11]:
train, test = train_test_split(energy_build_0, test_size=0.2)

In [12]:
ish_data = train[['hour', 'air_temperature', 'dew_temperature', 'sea_level_pressure', 'wind_speed', 'cloud_coverage']].to_numpy()

In [13]:
reg = linear_model.LinearRegression()
reg.fit(ish_data, train['meter_reading'].to_list())

LinearRegression()

In [14]:
print(reg.coef_)

[-0.8348427   5.37043316  8.51688764  4.83150057 -2.60656031 -1.07430021]


In [18]:
ish_test = test[['hour', 'air_temperature', 'dew_temperature', 'sea_level_pressure', 'wind_speed', 'cloud_coverage']].to_numpy()

In [22]:
from sklearn.metrics import mean_squared_error

In [23]:
print(mean_squared_error(reg.predict(ish_test),test['meter_reading'].to_list()))

9080.41425187495


In [24]:
from sklearn.metrics import mean_squared_log_error

In [25]:
print(mean_squared_log_error(abs(reg.predict(ish_test)),test['meter_reading'].to_list()))

8.274390681687134
